## **Code : Add variable - holiday, weekend**
- Writer : Donghyeon Kim
- Update : 2022.11.18.
- 대상
    1. 3kW Only Dataset
    2. 3kW + 300W + 18kW Dataset(Total)

<hr>

## **Prior Settings**

In [1]:
# 라이브러리 설정
import os
import pandas as pd
import numpy as np
import openpyxl
from datetime import datetime, date
import holidays

In [2]:
# 사용자 함수 호출
# get_project_root : 파일의 상위-상위 경로 호출
from pack_utils import get_project_root

<hr>

## **Data : 3kW Only**

### 1. Rawdata

In [3]:
# All Data
root = get_project_root()
folder_root = os.path.join(root, 'data_final_keei')
file_name = os.path.join(folder_root, 'Dataset Hours 20221118 3kW Only.xlsx')
df_all_hour = pd.read_excel(file_name, sheet_name=0)

# Filtering
# (1) Using Solar
df_all_hour_use = df_all_hour[df_all_hour.type == 'use']

# (2) Not Using Solar
df_all_hour_not = df_all_hour[df_all_hour.type == 'not']

In [4]:
df_all_hour.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 225299 entries, 0 to 225298
Data columns (total 27 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   가구번호                    225299 non-null  int64  
 1   연도                      225299 non-null  int64  
 2   월                       225299 non-null  int64  
 3   일                       225299 non-null  int64  
 4   시간                      225299 non-null  int64  
 5   시간2                     225299 non-null  int64  
 6   계절                      225299 non-null  object 
 7   설비용량(kW)                77148 non-null   object 
 8   발전량(kWh)                77148 non-null   float64
 9   전력소비량(kWh)              225299 non-null  float64
 10  수전전력량(kWh)              225299 non-null  float64
 11  잉여전력량(kWh)              77148 non-null   float64
 12  잉여전력량/발전량               77148 non-null   float64
 13  자가소비율                   77148 non-null   float64
 14  자가공급률               

### 2. 시간평균

In [5]:
# Average Data
root = get_project_root()
folder_root = os.path.join(root, 'data_final_keei')
file_name = os.path.join(folder_root, 'Dataset Hours 20221118 3kW Only.xlsx')
df_avg_hour = pd.read_excel(file_name, sheet_name=1)

In [6]:
df_avg_hour.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8760 entries, 0 to 8759
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   연도      8760 non-null   int64  
 1   월       8760 non-null   int64  
 2   일       8760 non-null   int64  
 3   시간      8760 non-null   int64  
 4   발전-미설치  0 non-null      float64
 5   발전-설치   8760 non-null   float64
 6   소비-미설치  8760 non-null   float64
 7   소비-설치   8760 non-null   float64
 8   수전-미설치  8760 non-null   float64
 9   수전-설치   8760 non-null   float64
 10  잉여-미설치  0 non-null      float64
 11  잉여-설치   8760 non-null   float64
dtypes: float64(8), int64(4)
memory usage: 821.4 KB


<hr>

### **주말 및 공휴일 체크 함수**

#### 주말 여부 체크

In [7]:
def check_weekend(date):
    weekday = date.weekday()
    if weekday <= 4:
        return False
    if weekday > 4:
        return True

#### 공휴일 여부 체크

In [8]:
kr_holidays = holidays.KR()
def check_holiday(date):
    mydate = date
    if mydate in kr_holidays:
        return True
    if mydate not in kr_holidays:
        return False

<hr>

### **주말 및 공휴일 추가**

#### 1. Rawdata : 사용자 별로 추가

In [14]:
user_name = df_all_hour.owner.unique().tolist()
weekend_result = []
holiday_result = []

for i in range(len(user_name)):
    df_all_hour_filter = df_all_hour[df_all_hour.owner == user_name[i]]
    u_year = df_all_hour_filter['연도'].unique()
    
    for y in u_year:
        day_filter1 = df_all_hour_filter[df_all_hour_filter['연도'] == y]
        u_month = day_filter1['월'].unique()

        for m in u_month:
            day_filter2 = day_filter1[day_filter1['월'] == m]
            u_day = day_filter2['일'].unique()

            for d in u_day:
                day_filter3 = day_filter2[day_filter2['일'] == d]
                u_hour = day_filter3['시간'].unique()
                
                for h in u_hour:
                    day_filter4 = day_filter3[day_filter3['시간'] == h]
                    
                    if check_weekend(date(y, m, d)) == True:
                        weekend_result.append(1)
                    else:
                        weekend_result.append(0)
                    
                    if check_holiday(date(y, m, d)) == True:
                        holiday_result.append(1)
                    else:
                        holiday_result.append(0)
                        
df_all_hour['weekend'] = weekend_result
df_all_hour['holiday'] = holiday_result

#### 2. 시간평균 : 날짜 별로 추가

In [16]:
u_year = df_avg_hour['연도'].unique()
weekend_result = []
holiday_result = []
    
for y in u_year:
    day_filter1 = df_avg_hour[df_avg_hour['연도'] == y]
    u_month = day_filter1['월'].unique()

    for m in u_month:
        day_filter2 = day_filter1[day_filter1['월'] == m]
        u_day = day_filter2['일'].unique()

        for d in u_day:
            day_filter3 = day_filter2[day_filter2['일'] == d]
            u_hour = day_filter3['시간'].unique()
                
            for h in u_hour:
                day_filter4 = day_filter3[day_filter3['시간'] == h]
                    
                if check_weekend(date(y, m, d)) == True:
                    weekend_result.append(1)
                else:
                    weekend_result.append(0)
                    
                if check_holiday(date(y, m, d)) == True:
                    holiday_result.append(1)
                else:
                    holiday_result.append(0)

df_avg_hour['weekend'] = weekend_result
df_avg_hour['holiday'] = holiday_result

<hr>

### **엑셀 저장**

In [18]:
xlsx_name = os.path.join(folder_root, '3kW only result1.xlsx')
df_all_hour.to_excel(xlsx_name, sheet_name='RawData', index=False)

In [19]:
xlsx_name2 = os.path.join(folder_root, '3kW only result2.xlsx')
df_avg_hour.to_excel(xlsx_name2, sheet_name='시간평균', index=False)

<hr>

## **Data : 3kW + 300W + 18kW**

### 1. Rawdata

In [20]:
# All Data
root = get_project_root()
folder_root = os.path.join(root, 'data_final_keei')
file_name = os.path.join(folder_root, 'Datast Hours 20221118 3kW 300W 18kW.xlsx')
df_all_hour = pd.read_excel(file_name, sheet_name=0)

# Filtering
# (1) Using Solar
df_all_hour_use = df_all_hour[df_all_hour.type == 'use']

# (2) Not Using Solar
df_all_hour_not = df_all_hour[df_all_hour.type == 'not']

In [21]:
df_all_hour.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 242540 entries, 0 to 242539
Data columns (total 26 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   가구번호                    242540 non-null  int64  
 1   연도                      242540 non-null  int64  
 2   월                       242540 non-null  int64  
 3   일                       242540 non-null  int64  
 4   시간                      242540 non-null  int64  
 5   계절                      242540 non-null  object 
 6   설비용량(kW)                94389 non-null   object 
 7   발전량(kWh)                94389 non-null   float64
 8   전력소비량(kWh)              242540 non-null  float64
 9   수전전력량(kWh)              242540 non-null  float64
 10  잉여전력량(kWh)              94389 non-null   float64
 11  잉여전력량/발전량               94389 non-null   float64
 12  자가소비율                   94389 non-null   float64
 13  자가공급률                   94389 non-null   float64
 14  temperature         

### 2. 시간평균

In [22]:
# Average Data
root = get_project_root()
folder_root = os.path.join(root, 'data_final_keei')
file_name = os.path.join(folder_root, 'Datast Hours 20221118 3kW 300W 18kW.xlsx')
df_avg_hour = pd.read_excel(file_name, sheet_name=1)

In [23]:
df_avg_hour.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8760 entries, 0 to 8759
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   연도      8760 non-null   int64  
 1   월       8760 non-null   int64  
 2   일       8760 non-null   int64  
 3   시간      8760 non-null   int64  
 4   발전-미설치  0 non-null      float64
 5   발전-설치   8760 non-null   float64
 6   소비-미설치  8760 non-null   float64
 7   소비-설치   8760 non-null   float64
 8   수전-미설치  8760 non-null   float64
 9   수전-설치   8760 non-null   float64
 10  잉여-미설치  0 non-null      float64
 11  잉여-설치   8760 non-null   float64
dtypes: float64(8), int64(4)
memory usage: 821.4 KB


<hr>

### **주말 및 공휴일 추가**

#### 1. Rawdata : 사용자 별로 추가

In [25]:
user_name = df_all_hour.owner.unique().tolist()
weekend_result = []
holiday_result = []

for i in range(len(user_name)):
    df_all_hour_filter = df_all_hour[df_all_hour.owner == user_name[i]]
    u_year = df_all_hour_filter['연도'].unique()
    
    for y in u_year:
        day_filter1 = df_all_hour_filter[df_all_hour_filter['연도'] == y]
        u_month = day_filter1['월'].unique()

        for m in u_month:
            day_filter2 = day_filter1[day_filter1['월'] == m]
            u_day = day_filter2['일'].unique()

            for d in u_day:
                day_filter3 = day_filter2[day_filter2['일'] == d]
                u_hour = day_filter3['시간'].unique()
                
                for h in u_hour:
                    day_filter4 = day_filter3[day_filter3['시간'] == h]
                    
                    if check_weekend(date(y, m, d)) == True:
                        weekend_result.append(1)
                    else:
                        weekend_result.append(0)
                    
                    if check_holiday(date(y, m, d)) == True:
                        holiday_result.append(1)
                    else:
                        holiday_result.append(0)
                        
df_all_hour['weekend'] = weekend_result
df_all_hour['holiday'] = holiday_result

#### 2. 시간평균 : 날짜 별로 추가

In [26]:
u_year = df_avg_hour['연도'].unique()
weekend_result = []
holiday_result = []
    
for y in u_year:
    day_filter1 = df_avg_hour[df_avg_hour['연도'] == y]
    u_month = day_filter1['월'].unique()

    for m in u_month:
        day_filter2 = day_filter1[day_filter1['월'] == m]
        u_day = day_filter2['일'].unique()

        for d in u_day:
            day_filter3 = day_filter2[day_filter2['일'] == d]
            u_hour = day_filter3['시간'].unique()
                
            for h in u_hour:
                day_filter4 = day_filter3[day_filter3['시간'] == h]
                    
                if check_weekend(date(y, m, d)) == True:
                    weekend_result.append(1)
                else:
                    weekend_result.append(0)
                    
                if check_holiday(date(y, m, d)) == True:
                    holiday_result.append(1)
                else:
                    holiday_result.append(0)

df_avg_hour['weekend'] = weekend_result
df_avg_hour['holiday'] = holiday_result

<hr>

### **엑셀 저장**

In [30]:
xlsx_name = os.path.join(folder_root, 'total result1.xlsx')
df_all_hour.to_excel(xlsx_name, sheet_name='RawData', index=False)

In [31]:
xlsx_name2 = os.path.join(folder_root, 'total result2.xlsx')
df_avg_hour.to_excel(xlsx_name2, sheet_name='시간평균', index=False)